# Water Quality CSV Transects joined by TIME/DATE 

In [1]:
import os
import pandas
from wq_gps_match import timestamp_match

In [2]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# example shp
shp = os.path.join(wd, "wq_gps_match", "examples", "Apr_2013", "SummaryFiles", "Apr2013_GPS", "Arc_Apr2013_PosnPnt.shp")

# example water quality
wq_file = os.path.join(wd, "wq_gps_match", "examples", "Apr_2013", "Arc_040213", "Arc_040213_WQ", "Arc_040213_wqt_sh_2.csv")

print(shp)
print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Apr_2013\SummaryFiles\Apr2013_GPS\Arc_Apr2013_PosnPnt.shp
C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Apr_2013\Arc_040213\Arc_040213_WQ\Arc_040213_wqt_sh_2.csv


Import CSV file from the MiniSonde as Pandas Dataframe

In [3]:
water_quality = timestamp_match.wq_from_csv(wq_file)
print(water_quality.head())

            Date_Time   Temp    pH SpCond   Sal   DO%    DO DEP25  PAR RPAR  \
1 2013-04-02 09:18:19  15.87  7.77   2809  1.51  87.9  7.15  0.23  529  610   
2 2013-04-02 09:18:20  15.87  7.77   2809  1.51  87.9  7.15  0.23  529  610   
3 2013-04-02 09:18:21  15.87  7.76   2809  1.51  88.0  7.17  0.21  527  610   
4 2013-04-02 09:18:22  15.87  7.76   2813  1.52  88.0  7.17  0.21  527  610   
5 2013-04-02 09:18:23  15.87  7.77   2813  1.52  88.1  7.17  0.22  536  610   

  TurbSC   CHL    CHL.1                WQ_SOURCE  
1   37.7  5.36  0.05174  Arc_040213_wqt_sh_2.csv  
2   37.7  5.36  0.05174  Arc_040213_wqt_sh_2.csv  
3   37.6  5.39  0.05200  Arc_040213_wqt_sh_2.csv  
4   37.6  5.39  0.05200  Arc_040213_wqt_sh_2.csv  
5   37.9  5.42  0.05216  Arc_040213_wqt_sh_2.csv  


In [4]:
shp_df = timestamp_match.shp2dataframe(shp)

print(shp_df.head())

       GPS_Date    GPS_Time           Date_Time              POINT_X  \
0  [2013, 4, 4]  08:18:47am 2013-04-04 08:18:47  -1.21675981849e+002   
1  [2013, 4, 4]  08:18:48am 2013-04-04 08:18:48  -1.21675986001e+002   
2  [2013, 4, 4]  08:18:49am 2013-04-04 08:18:49  -1.21675990421e+002   
3  [2013, 4, 4]  08:18:50am 2013-04-04 08:18:50  -1.21675994058e+002   
4  [2013, 4, 4]  08:18:51am 2013-04-04 08:18:51  -1.21675997428e+002   

              POINT_Y                               XY  \
0  3.82332247674e+001  [-121.675981849, 38.2332247674]   
1  3.82332196115e+001  [-121.675986001, 38.2332196115]   
2  3.82332143150e+001   [-121.675990421, 38.233214315]   
3  3.82332090390e+001   [-121.675994058, 38.233209039]   
4  3.82332039378e+001  [-121.675997428, 38.2332039378]   

                GPS_SOURCE  
0  Arc_Apr2013_PosnPnt.shp  
1  Arc_Apr2013_PosnPnt.shp  
2  Arc_Apr2013_PosnPnt.shp  
3  Arc_Apr2013_PosnPnt.shp  
4  Arc_Apr2013_PosnPnt.shp  


In [6]:
merged = timestamp_match.JoinByTimeStamp(water_quality, shp_df)
#print(merged)

matches = merged[0]

print('Dimensions - origninal WQ:  {},  Matches: {}'.format( water_quality.shape, matches.shape))

print('Number of rows different: {}'.format(water_quality.shape[0] - matches.shape[0]))

percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)

print('Percent Match: {} %'.format(percent_match))

Dimensions - origninal WQ:  (1921, 14),  Matches: (1921, 20)
Number of rows different: 0
Percent Match: 100.0 %


In [ ]:
# adjust date time by an hour to account for uncorrected daylight savings time

from datetime import timedelta


def dstadjustment(df, offset_hours):
    df2 = df.copy() # make a copy of data so original is not overwritten
    dstshift = lambda x: x + timedelta(hours=offset_hours)
    df2['Date_Time'] = df2['Date_Time'].map(dstshift)
    return df2



#orignal = timestamp_match.JoinByTimeStamp(water_quality, shp_df)
#percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)


#plus1 = dstadjustment(water_quality, 1)

offsets= {}

for i in [-1,-0.5, 0, 0.5,1]:
    print("Time offset: {} hour".format(i))
    
    # offset water quality
    off = dstadjustment(water_quality, i)
    
    # try joining by timestamp
    matches = timestamp_match.JoinByTimeStamp(off, shp_df)[0]
    
    # report the percentage matched
    percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)
    print("{} %".format(percent_match))
    
    # add to dict
    offsets[i]=percent_match

print(offsets)
highest_percent_offset = max(offsets, key=offsets.get)
print(highest_percent_offset)


offset_df = timestamp_match.dstadjustment(water_quality, highest_percent_offset)
offset_df.head()

# join using time stamps w/ exact matchjoined_data = timestamp_match.JoinByTimeStamp(offset_df, pts)

## transect_join_timestamp - batch function to combine transect water quality with GPS

In [ ]:
joined_data = timestamp_match.transect_join_timestamp(wq_file, shp)
print(joined_data)

In [ ]:
# save to shapefile

timestamp_match.main(wq_file, shp, "Example.shp")

# ZOOP CHL W points

In [ ]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# shapefile of the Chl sampling locations
station_shp = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_GPS", "040413_ZoopChlW.shp")
# water quality vertical profile for a Chl sampling point
station_wq_file = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_WQ", "Arc_040413_wqp_ca1.csv")

print(station_shp)
print(station_wq_file)

In [ ]:
# let's process that water quality at the stationary point
station_water_quality = timestamp_match.wq_from_csv(station_wq_file)
print(station_water_quality.head())

In [ ]:
# let's get the shapefile for the stationary points collected that day.

station_shp_df = timestamp_match.shp2dataframe(station_shp)

print(station_shp_df.head())

In [ ]:
# combine the GPS_date and GPS time fields to match a timestamp to use as join field
timestamp_match.addCombinedDateTime(station_shp_df, "GPS_Date", "GPS_Time")
print(station_shp_df.head())

In [ ]:
# for each row in the water quality dataframe join the station in if it's with xx minutes. 
station_gps_datetimes = station_shp_df["Date_Time"].tolist()
print(station_gps_datetimes)

# for each row find the closest time stamp


import datetime

ex = station_water_quality["Date_Time"][10]
print("example: {}".format(ex))



row_time = ex
print("Test: {}".format(row_time))


def my_minimum(iterable):
    iterable = iter(iterable)
    minimum = iterable.next()
    for i in iterable:
        if i < minimum:
            minimum = i
    return minimum

print(my_minimum(station_gps_datetimes))


def find_closest_time(row_time, list_of_potential_matches):
    iterable = iter(list_of_potential_matches)
    minimum = iterable.next()
    for t = iterable

    for sample_time in list_of_potential_matches:
        diff = (row_time - sample_time)
        
        if diff < 1:
            closest_time = sample_time
            gap = diff
        print(gap)
            
    # check if closest time is 1 hour        
    if gap == datetime.timedelta(hours=1):
        return "No Match"
    else:
        return closest_time

print(find_closest_time(row_time, station_gps_datetimes))